**Similitud del Coseno**
-----------------
-----------------
**Definicion**

La similitud del coseno es una métrica utilizada para medir la semejanza entre dos vectores en un espacio multidimensional. Esta herramienta es ampliamente utilizada en sistemas de recomendación, especialmente en métodos de filtrado colaborativo basado en ítems o usuarios. En un sistema de recomendacion, los usuarios o los items son los vectores cuya similitud trataremos de estimar

La similitud del coseno entre dos vectores 𝐴 y 𝐵 se define como el coseno del ángulo entre ellos. El valor resultante está en el rango [-1, 1], donde:
- 1 indica que los vectores son idénticos.
- 0 indica que los vectores son ortogonales (no tienen similitud).
- -1 indica que los vectores son opuestos.

La definición matematica formal es

$\text{sim}(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}$
---------------------------

$\mathbf{A} \cdot \mathbf{B}$ es el producto punto de los vectores $\mathbf{A}$ y $\mathbf{B}$ donde

- $\mathbf{A} \cdot \mathbf{B} = \sum_{i=1}^n A_i B_i$

$\|\mathbf{A}\|$ y $\|\mathbf{B}\|$ son las normas (longitudes) de los vectores $(\mathbf{A})$ y $(\mathbf{B})$, respectivamente, calculadas como:

- $\|\mathbf{A}\| = \sqrt{\sum_{i=1}^n A_i^2}$

- $\|\mathbf{B}\| = \sqrt{\sum_{i=1}^n B_i^2}$

Cargamos las librerias y bases de datos
---------------------

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

In [2]:
# Exploración de los datos
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# Crear la matriz de usuarios y películas
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_movie_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# Crear la matriz de calificaciones (usuarios x películas)
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')

# Extraer los vectores de calificaciones de los usuarios 4 y 5
user4_ratings = ratings_matrix.loc[4]
user5_ratings = ratings_matrix.loc[5]

# Encontrar las columnas donde ambos usuarios han dado calificaciones
common_ratings = ~user4_ratings.isnull() & ~user5_ratings.isnull()

# Filtrar las calificaciones comunes
user4_common = user4_ratings[common_ratings].values
user5_common = user5_ratings[common_ratings].values

# Mostrar los vectores de calificaciones filtrados
print("Vector de calificaciones del usuario 4 (filtrado):", user4_common)
print("Vector de calificaciones del usuario 5 (filtrado):", user5_common)

# Calcular la similitud del coseno entre los usuarios 4 y 5 en las calificaciones comunes
cosine_sim = cosine_similarity([user4_common], [user5_common])[0][0]
print(f"Similitud del coseno entre el usuario 4 y el usuario 5 (calificaciones comunes): {cosine_sim}")

# Calcular el ángulo entre los dos vectores
angle = np.arccos(cosine_sim)
angle_degrees = np.degrees(angle)
print(f"Ángulo entre los vectores de calificaciones (en radianes): {angle}")
print(f"Ángulo entre los vectores de calificaciones (en grados): {angle_degrees}")

Vector de calificaciones del usuario 4 (filtrado): [3. 3. 5. 3. 5. 1. 3. 5. 5. 4. 3. 5.]
Vector de calificaciones del usuario 5 (filtrado): [4. 5. 4. 5. 3. 5. 2. 4. 5. 4. 5. 3.]
Similitud del coseno entre el usuario 4 y el usuario 5 (calificaciones comunes): 0.9011373697672393
Ángulo entre los vectores de calificaciones (en radianes): 0.4484104366060839
Ángulo entre los vectores de calificaciones (en grados): 25.69202550714716


Caso simple
--------------
Vamos a tomar nuestra base de datos para obtener los k usuarios más similares al usuario 4.

Para comparar dos usuarios (filas) de la matriz, vamos a tomar en cuenta las peliculas (columnas) donde ambos hayan establecido una calificacion. Cuanto mas peliculas en comun vean dos usuarios, mas significativo es el resultado del calculo de la similitud entre ellos.

Vamos a extraer la lista de las peliculas que estos k usuarios han visto, para recomendarle esas peliculas al usuario 4. En la medida de lo posible, trataremos de no recomendarle peliculas al usuario 4 que ya haya calificado en el pasado

In [40]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Cargar los datos
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

# Crear la matriz de calificaciones (usuarios x películas)
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')


# Extraer el vector de calificaciones del usuario 4
a = 4
user4_ratings = ratings_matrix.loc[a]

# Calcular la similitud del coseno entre el usuario 4 y todos los demás usuarios
similarities = {}

for user in ratings_matrix.index:
    if user == a:
        continue
    
    user_ratings = ratings_matrix.loc[user]
    
    # Encontrar las columnas donde ambos usuarios han dado calificaciones
    common_ratings = ~user4_ratings.isnull() & ~user_ratings.isnull()
    
    if common_ratings.sum() < 4: # Solo vamos a incliuir usuarios que tengan mas de 4 peliculas en comun con el usuario target
        continue  # Si no hay suficientes calificaciones comunes, saltar este usuario
    
    # Filtrar las calificaciones comunes
    user4_common = user4_ratings[common_ratings].values
    user_common = user_ratings[common_ratings].values
    
    # Calcular la similitud del coseno entre el usuario 4 y el usuario actual
    cosine_sim = cosine_similarity([user4_common], [user_common])[0][0]
    similarities[user] = cosine_sim

# Ordenar los usuarios por similitud del coseno en orden descendente y tomar los 10 más similares
most_similar_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:10]

# Mostrar los 10 usuarios más similares
print(f"Los 10 usuarios más similares al usuario {a}:")
for user, similarity in most_similar_users:
    print(f"Usuario {user} con similitud del coseno de {similarity}")

# Obtener la lista de películas que el usuario 4 ha calificado
user4_movies = set(user4_ratings.dropna().index)

# Obtener la lista de películas que los usuarios más similares han calificado
similar_users_movies = set()
for user, _ in most_similar_users:
    user_movies = set(ratings_matrix.loc[user].dropna().index)
    similar_users_movies |= user_movies

# Filtrar las películas para excluir las que el usuario 4 ya ha visto
recommended_movies = similar_users_movies - user4_movies

# Seleccionar una de las películas restantes (si existe alguna)
if recommended_movies:
    for i, recommended_movie in enumerate(recommended_movies):
        if i >= 10:
            break
        recommended_movie_title = movies[movies['movieId'] == recommended_movie]['title'].values[0]
        print(f"Película recomendada para el usuario 4: {recommended_movie_title} (ID: {recommended_movie})")
else:
    print("No hay películas que cumplan con los criterios especificados.")


Los 10 usuarios más similares al usuario 4:
Usuario 502 con similitud del coseno de 0.9961071065057698
Usuario 397 con similitud del coseno de 0.9943133448792364
Usuario 44 con similitud del coseno de 0.9908807141567176
Usuario 521 con similitud del coseno de 0.9884197914907037
Usuario 482 con similitud del coseno de 0.985135423826942
Usuario 218 con similitud del coseno de 0.9844951849708404
Usuario 401 con similitud del coseno de 0.9836607789011884
Usuario 128 con similitud del coseno de 0.9834404237954975
Usuario 194 con similitud del coseno de 0.9833993487410667
Usuario 114 con similitud del coseno de 0.9833783437888834
Película recomendada para el usuario 4: Toy Story (1995) (ID: 1)
Película recomendada para el usuario 4: Jumanji (1995) (ID: 2)
Película recomendada para el usuario 4: Grumpier Old Men (1995) (ID: 3)
Película recomendada para el usuario 4: Mad Max: Fury Road (2015) (ID: 122882)
Película recomendada para el usuario 4: Father of the Bride Part II (1995) (ID: 5)
Pelícu

Verificamos los resultados obtenidos
-----------------

In [52]:
def has_user_seen_movie(user_id, movie_id, ratings_matrix):
    """
    Verifica si un usuario ha visto una determinada película.
    
    Parámetros:
    user_id (int): ID del usuario.
    movie_id (int): ID de la película.
    ratings_matrix (pd.DataFrame): Matriz de calificaciones (usuarios x películas).
    
    Retorna:
    bool: True si el usuario ha visto la película, False en caso contrario.
    """
    if user_id in ratings_matrix.index and movie_id in ratings_matrix.columns:
        return not pd.isna(ratings_matrix.loc[user_id, movie_id])
    else:
        return False

# Ejemplo de uso
user_id = 4
movie_id = 1

if has_user_seen_movie(user_id, movie_id, ratings_matrix):
    print(f"El usuario {user_id} ha visto la película con ID {movie_id}.")
else:
    print(f"El usuario {user_id} no ha visto la película con ID {movie_id}.")


El usuario 4 no ha visto la película con ID 1.
